In [ ]:
!pip install ultralytics


In [ ]:
!pip install pytube opencv-python


In [ ]:
!pip install paddlepaddle-gpu


In [ ]:
!pip install paddleocr

In [ ]:
# gdown 설치 (캐글 환경에 설치되어 있지 않은 경우)
!pip install gdown


In [ ]:
import gdown


# 파일 ID로 다운로드
file_id = '1CArby0IAL63i3rYjlW2fbE-01z3oyNm9'
gdown.download(f'https://drive.google.com/uc?id={file_id}', 'video.mp4', quiet=False)


In [ ]:

import cv2
import numpy as np
from ultralytics import YOLO
from paddleocr import PaddleOCR
from IPython.display import display, Image
import matplotlib.pyplot as plt


In [ ]:

# 첫 번째 YOLOv8 모델 로드
model1 = YOLO('yolov8n.pt')

# 두 번째 YOLOv8 모델 로드
model2 = YOLO('/kaggle/input/yolov8_license_plate/pytorch/default/1/yolov8_license_plate.pt')


In [ ]:

# OCR 모델 초기화
ocr = PaddleOCR(use_angle_cls=True, lang='korean')


In [ ]:
# 비디오 파일 경로
video_path = 'video.mp4'


In [ ]:
cap = cv2.VideoCapture(video_path)

In [ ]:

frame_rate = cap.get(cv2.CAP_PROP_FPS)  # 프레임 속도 가져오기
process_interval = int(frame_rate)  # 1초 마다 처리


In [ ]:

# 차량 클래스 ID 목록
vehicle_classes = [2, 3, 7]


In [ ]:
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    if frame_count % process_interval == 0:
        # 차량 탐지
        vehicle_results = model1(frame)
        
        for vehicle_result in vehicle_results:
            boxes = vehicle_result.boxes
            for box in boxes:
                cls = int(box.cls[0])
                if cls in vehicle_classes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    
                    # 차량 영역 크롭
                    cropped_image = frame[y1:y2, x1:x2]
                    
                    # 번호판 탐지
                    license_plate_results = model2(cropped_image)
                    
                    for license_plate_result in license_plate_results:
                        license_plate_boxes = license_plate_result.boxes
                        for license_plate_box in license_plate_boxes:
                            x1_c, y1_c, x2_c, y2_c = map(int, license_plate_box.xyxy[0])
                            
                            # 번호판 영역 표시
                            cv2.rectangle(cropped_image, (x1_c, y1_c), (x2_c, y2_c), (255, 0, 0), 2)
                            
                            # OCR 수행
                            license_plate_image = cropped_image[y1_c:y2_c, x1_c:x2_c]
                            ocr_result = ocr.ocr(license_plate_image, cls=True)
                            
                            # OCR 결과 출력 및 이미지에 표시
                            if ocr_result:
                                text = ""
                                for line in ocr_result:
                                    if line:
                                        for word_info in line:
                                            text += word_info[1][0] + " "
                                
                                # 번호판 텍스트를 이미지에 표시
                                cv2.putText(cropped_image, text, (x1_c, y1_c - 10), 
                                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                                
                                # 콘솔에 번호판 텍스트 출력
                                print(f'License Plate: {text.strip()}')
                    
                    # 크롭된 이미지를 원본 프레임에 다시 붙이기
                    frame[y1:y2, x1:x2] = cropped_image
        
        # 결과를 이미지 파일로 저장
        output_path = f'output_frame_{frame_count}.jpg'
        cv2.imwrite(output_path, frame)
        
        # 저장된 이미지를 노트북에 표시
        display(Image(filename=output_path))
        
        # 잠시 대기 (필요에 따라 조절)
        plt.pause(1)

    frame_count += 1

# 자원 해제
cap.release()
